In [1]:
import pandas as pd
import numpy as np

from src.language_processing2 import processing_text
from src.language_processing2 import vectorize_text
from src.language_processing2 import vectorize_desc
from src.modeling import test_model
from src.modeling import test_model2
from src.modeling import test_model3
from src.modeling import test_model4

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split,KFold

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
def test_model6(data,column,TFIDF_column,estimators,dep):
    data2 = data.sample(frac=1)
    data2_training = data2[:16000]
    data2_testing = data2[16000:]
    y = data2_training[column]
    X = data2_training[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count','favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none'] + TFIDF_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    model = GradientBoostingClassifier(subsample=0.5, learning_rate=0.1, n_estimators=estimators,max_depth=dep)
    kf = KFold(n_splits=5, shuffle=True)

    ll_performance = []
    auc_performance = []
    acc_performance = []
#kfold split on X_ (which is X_train of len 110)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        y_pred = model.predict_proba(X_test)
        log_ll = log_loss(y_test, y_pred)
        ll_performance.append(log_ll)
        auc = roc_auc_score(y_test,y_predict)
        auc_performance.append(auc)
        acc = accuracy_score(y_test,y_predict)
        acc_performance.append(acc)
        
        
    print("\n" + "Score summary for initial test (first 80% of data)" + "\n")
    print("\n" + "Accuracy score of this model is" + "\n")
    print(np.mean(acc_performance))
    print("\n" + "Log-Loss score of this model is" + "\n")
    print(np.mean(ll_performance))
    print("\n" + "AUC score of this model is" + "\n")
    print(np.mean(auc_performance))
    print("\n")

    y = data2_testing[column]
    X = data2_testing[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count','favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none'] + TFIDF_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    #model = model()
    kf = KFold(n_splits=5, shuffle=True)

    ll_performance = []
    auc_performance = []
    acc_performance = []
#kfold split on X_ (which is X_train of len 110)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        y_pred = model.predict_proba(X_test)
        log_ll = log_loss(y_test, y_pred)
        ll_performance.append(log_ll)
        auc = roc_auc_score(y_test,y_predict)
        auc_performance.append(auc)
        acc = accuracy_score(y_test,y_predict)
        acc_performance.append(acc)
        
        
    print("\n" + "Score summary for final test (last 20% of data)" + "\n")
    print("\n" + "Accuracy score of this model is" + "\n")
    print(np.mean(acc_performance))
    print("\n" + "Log-Loss score of this model is" + "\n")
    print(np.mean(ll_performance))
    print("\n" + "AUC score of this model is" + "\n")
    print(np.mean(auc_performance))
    #print("\n" + "Coefficients of this model are" + "\n")
    #print(model.coef_)
    #print("\n" + "Most effective predictor was" + "\n")
    return None

In [5]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,700)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,700)
test_model6(df,'spam_marketing',col_name_lst1+col_name_lst2,300,2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9530625

Log-Loss score of this model is

0.12743012650190894

AUC score of this model is

0.6828016362934268



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9460817369808936

Log-Loss score of this model is

0.161360256625824

AUC score of this model is

0.630423425336059


In [6]:
depth_candidate = list(range(2,10,2))

In [7]:
depth_candidate

[2, 4, 6, 8]